### Objective :
To build a Feed Forward Network for eMNIST Classification in Pytorch in about 5 epochs.

### Parameters :
1. Number of parameters used in the model ( lower the better)
2. Validation data accuracy (higher the better)

**Solution** :

Importing necessary Libraries

In [2]:
import torch #You have to use python 3.12 or lower :( 
from torch import nn
import torch.nn.functional as F
from torchvision import datasets,transforms
from torch import optim

Uploading the train and test data using dataloaders

In [3]:
transform=transforms.Compose([transforms.ToTensor()])

trainset = datasets.EMNIST(root='~/.pytorch/eMNIST_data/',  split='letters',  train=True, download=True, transform=transform)
testset = datasets.EMNIST(root='~/.pytorch/eMNIST_data/',  split='letters',  train=False, download=True, transform=transform)
trainset.targets -= 1 # making sure that the classes are 0 indexed
testset.targets -= 1

trainloader=torch.utils.data.DataLoader(trainset,batch_size=100,shuffle=True,num_workers=0)

testloader=torch.utils.data.DataLoader(testset,batch_size=100,shuffle=True,num_workers=0)

Defining the Neural Network Architecture

In [4]:
#this is what you would implement

class Network(nn.Module):
    def __init__(self):
        
        super().__init__()
        feature1 = 40
        feature2 = 160
        dropout = 0.5
        # Convolutional layers
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=feature1, kernel_size=5, stride=1, padding='same'),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=feature1, out_channels=feature2, kernel_size=5, stride=1, padding='same'),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.flattened_size = 49 * feature2
        
        # Fully connected layers
        self.fcon1 = nn.Sequential(
            nn.Linear(self.flattened_size, 100), 
            nn.ReLU()
        )
        self.fcon2 = nn.Sequential(
            nn.Linear(100, 50),
            nn.ReLU()
        )
        self.fcon3 = nn.Sequential(
            nn.Linear(50, 50),
            nn.ReLU()
        )
        self.output_layer = nn.Linear(50, 26)

        self.dropout = nn.Dropout(p=dropout)


    def forward(self, x):

        '''
        x = F.log_softmax(<last layers's output>, dim=1)
        '''
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fcon1(x)
        x = self.dropout(x)
        x = self.fcon2(x)
        x = self.dropout(x)
        x = self.fcon3(x)
        x = self.dropout(x)
        x = F.log_softmax(x, dim=1)
        return x

model=Network()


optimizer = optim.Adam(model.parameters(), lr=0.001)




criterion=nn.NLLLoss() #negative log likelihood loss

## I was getting an error when training the data, 
### I asked ChatGPT how to fix it, that's why line 7 is commented out. 

"The convolutional layers in your model expect input tensors of a specific shape, typically [batch_size, channels, height, width], but the reshaping step images = images.view(images.shape[0], -1) flattens the images into a 2D tensor of shape [batch_size, num_features]."

In [5]:
epochs=5
train_losses,test_losses=[],[]
for e in range(epochs):
    running_loss=0
    for images,labels in trainloader:
        optimizer.zero_grad()
        #images=images.view(images.shape[0],-1)
        log_ps=model(images)
        loss=criterion(log_ps,labels) # a single value for ex 2.33
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.shape[0] ## (2.33*64 + 2.22*64 + 2.12*33) / 138

    else:
        test_loss=0
        accuracy=0

        with torch.no_grad():
            model.eval()
            for images,labels in testloader:
                log_ps=model(images)
                test_loss+=criterion(log_ps,labels) *images.shape[0]
                ps=torch.exp(log_ps)
                top_p,top_class=ps.topk(1,dim=1)
                equals=top_class==labels.view(*top_class.shape)
                accuracy+=torch.sum(equals).item()
        model.train()
        train_losses.append(running_loss/len(trainloader.dataset))
        test_losses.append(test_loss.item()/len(testloader.dataset))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader.dataset)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(testloader.dataset)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader.dataset)))

Epoch: 1/5..  Training Loss: 3.047..  Test Loss: 1.526..  Test Accuracy: 0.874
Epoch: 2/5..  Training Loss: 2.624..  Test Loss: 1.264..  Test Accuracy: 0.902
Epoch: 3/5..  Training Loss: 2.506..  Test Loss: 1.115..  Test Accuracy: 0.914
Epoch: 4/5..  Training Loss: 2.442..  Test Loss: 1.009..  Test Accuracy: 0.919
Epoch: 5/5..  Training Loss: 2.391..  Test Loss: 0.905..  Test Accuracy: 0.926


Calculating the total number of parameters

In [6]:
print("Our model: \n\n", model, '\n')

pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

Our model: 

 Network(
  (conv1): Sequential(
    (0): Conv2d(1, 40, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(40, 160, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fcon1): Sequential(
    (0): Linear(in_features=7840, out_features=100, bias=True)
    (1): ReLU()
  )
  (fcon2): Sequential(
    (0): Linear(in_features=100, out_features=50, bias=True)
    (1): ReLU()
  )
  (fcon3): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): ReLU()
  )
  (output_layer): Linear(in_features=50, out_features=26, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
) 



954226